In [333]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime

In [325]:
def get_stations(url):
    response = requests.get(url)
    webpage = BeautifulSoup(response.content, 'lxml')
    html = webpage.find_all('div', class_='stName')
    stations = {}
    for tags in html:
        station_id = tags.find('span', class_='stName').get('mb_sd_id')
        station_name = tags.find('span', class_='stName').get_text()
        stations[station_id] = re.sub(r'\n', ' ', station_name)
    return stations

def get_key(dictionary, value):
    key = [k for k, v in dictionary.items() if v == value][0]
    return key

def get_data_key(from_station_id, to_station_id):
    table = requests.post('https://metrobook.ru/kcmsajax.php',
                          data={
                              'mod': 'metrobook',
                              'oper': 'getShortestPath',
                              'sdid1': from_station_id,
                              'sdid2': to_station_id,
                              'mid': '2',
                              'whatToMinimize': '0'
                          }).json()
    
    from_station_name = stations.get(from_station_id)
    to_station_name = stations.get(to_station_id)
    transit_time = datetime.timedelta(seconds=table['table']['transitTime'])
    total_time = datetime.timedelta(seconds=table['table']['totalTime'])
    data_key = [from_station_name, to_station_name, total_time]
    return data_key

In [330]:
raw_data = []
from_station = 'Кутузовская'
to_station = 'Тульская'
limit_minutes = False

FILENAME = 'data.csv'

stations = get_stations('https://metrobook.ru')
from_station_id = get_key(stations, from_station)
limit_time = [datetime.timedelta(minutes = limit_minutes)]
stations_list = list(stations.keys())

if not to_station:
    for st_id in stations_list:
        to_station_id = str(st_id)
        
        if from_station_id != to_station_id:
            data_key = get_data_key(from_station_id, to_station_id)
            raw_data.append(data_key)
else:
    to_station_id = get_key(stations, to_station)
    data_key = get_data_key(from_station_id, to_station_id)
    raw_data.append(data_key)

if limit_minutes != False:
    limit_time = datetime.timedelta(minutes = limit_minutes)
    data = [x for x in raw_data if x[-1] <= limit_time]
else:
    data = raw_data

In [331]:
data = pd.DataFrame(data, columns=['FROM_STATION', 'TO_STATION', 'TOTAL_TIME'])

In [332]:
data

,FROM_STATION,TO_STATION,TOTAL_TIME
0,Кутузовская,Тульская,0 days 00:21:00


In [329]:
data.to_csv(FILENAME, index=False, encoding='utf-8')